## Data
- 첨부된 ko-en-en.parse.syn은 330,974 한국어 문장에 대응되는 영어문장이 품사와 구문분석이 되어 있는 파일이고 ko-en-ko.parse.syn은 이에 대응되는 한국어 문장이 형태소와 구문분석이 되어 있는 파일이다.

(ROOT (S (NP (NNP Flight) (NNP 007)) (VP (MD will) (VP (VB stay) (PP (IN on) (NP (NP (DT the) (NN ground)) (PP (IN for) (NP (CD one) (NN hour))))))) (. .)))


<id 1>
<sent 1>
1       2       NP      777/SN
2       6       NP_SBJ  항공편/NNG|은/JX
3       4       NP      1/SN|시간/NNG
4       6       NP_AJT  동안/NNG
5       6       NP_AJT  지상/NNG|에/JKB
6       7       VP      머물/VV|게/EC
7       0       VP      되/VV|ㅂ니다/EF|./SF
</sent>
</id>

- 이 두 파일을 프로세싱하여 한-영 병행 데이터로 만들고 이를 학습 및 테스트 데이터로 사용한다.
- Hint: 구조화된 데이터를 프로세싱하기 위해서는 nltk의 모듈을 사용할 수 있다.

- 한국어 형태소 분석된 단위를 어절별로 결합할 수 있고, 분석된 채로 그대로 사용할 수도 있다.
- 두 언어의 어순을 비슷하게 데이터를 만들어 학습할 수도 있고, 번역의 성능을 높이기 위해 다양한 형태로 재구조화 할 수 있다.

# 1. Helsinki_NLP_opus_mt_ko_en

## 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 파일 불러오기

In [ ]:
PATH = "/content/drive/MyDrive/"

In [ ]:
ko_path = PATH+"ko-en.ko.parse"
en_path = PATH+"ko-en.en.parse.syn"

In [ ]:
import pandas as pd

In [ ]:
ko_lines = ""
with open(ko_path, "r") as ko_file:
    for line in ko_file.readlines():
        ko_lines += line

In [ ]:
with open(en_path, "r") as en_file:
    en_lines = en_file.readlines()

## 데이터 전처리 (깔끔한 문장의 리스트로 만들기)

In [ ]:
from nltk import Tree

In [ ]:
# nltk의 Tree 모듈을 사용하여 필요한 정보 추출
full_en_text_list = []
for line in en_lines:
    sent = ""
    t = Tree.fromstring(line)
    for token in t.leaves():
      sent += token + ' '
    full_en_text_list.append(sent)

In [ ]:
full_en_text_list[:10]

['Flight 007 will stay on the ground for one hour . ',
 'Flight 017 will stay on the ground for three hours . ',
 "I need 1,000 dollars in traveler 's checks . ",
 'The official exchange rate is around 1,250 Won . ',
 'Please give me three hundred dollar bills and twenty dollar bills for the rest . ',
 'Can I have one hundred dollar bill and four fifty dollar bills ? ',
 'Do you have change for $ 100 ? ',
 "I 'd like to change 100 dollars . ",
 "I 'd like to change $ 100 . ",
 'Change 100 dollars . ']

In [ ]:
# 한 문장씩 나눈 리스트 만들기
ko_list = ko_lines.split("</id>")
for i in range(len(ko_list)):
    ko_list[i] = ko_list[i].split("\n")

ko_list[0]

['<id 1>',
 '<sent 1>',
 '1\t2\tNP\t777/SN',
 '2\t6\tNP_SBJ\t항공편/NNG|은/JX',
 '3\t4\tNP\t1/SN|시간/NNG',
 '4\t6\tNP_AJT\t동안/NNG',
 '5\t6\tNP_AJT\t지상/NNG|에/JKB',
 '6\t7\tVP\t머물/VV|게/EC',
 '7\t0\tVP\t되/VV|ㅂ니다/EF|./SF',
 '</sent>',
 '']

In [ ]:
# 필요한 정보만 추출
import re

pattern = r"[가-힣ㄱ-ㅎ]+|[0-9]+(?=\/SN)"

for i in range(len(ko_list)):
    for j in range(len(ko_list[i])):
        ko_list[i][j] = re.findall(pattern, ko_list[i][j])

In [ ]:
ko_list[:10]

[[[],
  [],
  ['777'],
  ['항공편', '은'],
  ['1', '시간'],
  ['동안'],
  ['지상', '에'],
  ['머물', '게'],
  ['되', 'ㅂ니다'],
  [],
  []],
 [[],
  [],
  [],
  [],
  ['777'],
  ['항공편', '은'],
  ['3', '시간'],
  ['동안'],
  ['지상', '에'],
  ['있', '겠', '습니다'],
  [],
  []],
 [[],
  [],
  [],
  [],
  ['1', '000', '달러'],
  ['여행자', '수표', '가'],
  ['필요', '하', 'ㅂ니다'],
  [],
  []],
 [[], [], [], [], ['1', '250', '원', '이'], ['공식'], ['환율', '이', 'ㅂ니다'], [], []],
 [[],
  [],
  [],
  [],
  ['100', '달러'],
  ['3', '장', '과'],
  ['나머지', '는'],
  ['20', '달러', '권', '으로'],
  ['주', '시', 'ㅂ시오'],
  [],
  []],
 [[],
  [],
  [],
  [],
  ['100', '달러'],
  ['한'],
  ['장', '과'],
  ['50', '달러'],
  ['4', '장', '으로'],
  ['바꾸', '어'],
  ['주', '시', '겠', '어요'],
  [],
  []],
 [[],
  [],
  [],
  [],
  ['100', '달러', '를'],
  ['바꾸', '어'],
  ['주', '시', '겠', '어요'],
  [],
  []],
 [[], [], [], [], ['100', '달러', '만'], ['바꾸', '어'], ['주', '시', '어요'], [], []],
 [[],
  [],
  [],
  [],
  ['100', '달러', '만'],
  ['환전'],
  ['좀'],
  ['하', '아'],
  ['주', '시', '어요'],
  []

In [ ]:
# 문장으로 만들어서 리스트에 넣어주기
ko_text_list = []

for i in range(len(ko_list)-1): # id로 split 해서 1개 추가로 생긴 부분 빼줌
    full_sent = ""
    for j in range(len(ko_list[i])):
        if ko_list[i][j]:
            for token in ko_list[i][j]:
                full_sent += token + ' '

    ko_text_list.append(full_sent)

In [ ]:
ko_text_list[:10]

['777 항공편 은 1 시간 동안 지상 에 머물 게 되 ㅂ니다 ',
 '777 항공편 은 3 시간 동안 지상 에 있 겠 습니다 ',
 '1 000 달러 여행자 수표 가 필요 하 ㅂ니다 ',
 '1 250 원 이 공식 환율 이 ㅂ니다 ',
 '100 달러 3 장 과 나머지 는 20 달러 권 으로 주 시 ㅂ시오 ',
 '100 달러 한 장 과 50 달러 4 장 으로 바꾸 어 주 시 겠 어요 ',
 '100 달러 를 바꾸 어 주 시 겠 어요 ',
 '100 달러 만 바꾸 어 주 시 어요 ',
 '100 달러 만 환전 좀 하 아 주 시 어요 ',
 '100 달러 만 환전 하 아 주 시 어요 ']

In [ ]:
print(ko_text_list[:10])
print(full_en_text_list[:10])

print(len(ko_text_list))
print(len(full_en_text_list))

['777 항공편 은 1 시간 동안 지상 에 머물 게 되 ㅂ니다 ', '777 항공편 은 3 시간 동안 지상 에 있 겠 습니다 ', '1 000 달러 여행자 수표 가 필요 하 ㅂ니다 ', '1 250 원 이 공식 환율 이 ㅂ니다 ', '100 달러 3 장 과 나머지 는 20 달러 권 으로 주 시 ㅂ시오 ', '100 달러 한 장 과 50 달러 4 장 으로 바꾸 어 주 시 겠 어요 ', '100 달러 를 바꾸 어 주 시 겠 어요 ', '100 달러 만 바꾸 어 주 시 어요 ', '100 달러 만 환전 좀 하 아 주 시 어요 ', '100 달러 만 환전 하 아 주 시 어요 ']
['Flight 007 will stay on the ground for one hour . ', 'Flight 017 will stay on the ground for three hours . ', "I need 1,000 dollars in traveler 's checks . ", 'The official exchange rate is around 1,250 Won . ', 'Please give me three hundred dollar bills and twenty dollar bills for the rest . ', 'Can I have one hundred dollar bill and four fifty dollar bills ? ', 'Do you have change for $ 100 ? ', "I 'd like to change 100 dollars . ", "I 'd like to change $ 100 . ", 'Change 100 dollars . ']
330974
330974


## 데이터셋 구축

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 22.1 MB/s eta 0:00:00


In [ ]:
from datasets import DatasetDict, Dataset, Features, Value

data_dict = {
    'id': range(1, len(ko_text_list) + 1),
    'translation': [{'ko': ko, 'en': en} for ko, en in zip(ko_text_list, full_en_text_list)]
}

features = Features({
    'id': Value('int32'),
    'translation': {'ko': Value('string'), 'en': Value('string')}
})

raw_datasets = Dataset.from_dict(data_dict, features=features)
dataset = DatasetDict({'train': raw_datasets})

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 330974
    })
})


In [ ]:
split_datasets = dataset["train"].train_test_split(train_size=0.8, test_size=0.1, seed=20)

dataset["train"] = split_datasets["train"]
dataset["validation"] = split_datasets["test"].train_test_split(train_size=0.5, seed=20)["train"]
dataset["test"] = split_datasets["test"].train_test_split(train_size=0.5, seed=20)["test"]

dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 264779
    })
    validation: Dataset({
        features: ['id', 'translation'],
        num_rows: 16549
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 16549
    })
})

## 데이터 전처리 (tokenize)

In [ ]:
!pip install transformers==4.28.0 # PartialState Error를 방지하기 위해 4.28.0 버전 사용
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 79.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is i

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "Helsinki-NLP/opus-mt-ko-en"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
dataset['train'][0]['translation']['ko'], dataset['train'][0]['translation']['en']

('즉석 에서 투표 하 자 ', "Let 's vote before we speak another word . ")

In [ ]:
max_src_length = 128
max_trg_length = 128

def tokenize_function(examples):
    src = [ex["ko"] for ex in examples["translation"]]
    trg = [ex["en"] for ex in examples["translation"]]
    model_inputs = tokenizer(src, text_target=trg, max_length=max_src_length, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(trg, max_length=max_trg_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [ ]:
tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

Map:   0%|          | 0/264779 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/16549 [00:00<?, ? examples/s]

Map:   0%|          | 0/16549 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 264779
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 16549
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 16549
    })
})

## Model

In [ ]:
import torch

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

## Data Collation

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [ ]:
batch["labels"]

tensor([[   16,  7628,    18,   867,  1220,    49, 53552,  1479,   639,  9079,
          1479, 11927,     9,     2,     0],
        [  417,   892,     6,     4,  1342,  4416,    61,    31,  2690,     5,
             4, 13596,     9,     2,     0]])

In [ ]:
batch["decoder_input_ids"]

tensor([[65000,    16,  7628,    18,   867,  1220,    49, 53552,  1479,   639,
          9079,  1479, 11927,     9,     2],
        [65000,   417,   892,     6,     4,  1342,  4416,    61,    31,  2690,
             5,     4, 13596,     9,     2]])

In [ ]:
for i in range(1, 3):
    print(tokenized_datasets["train"][i]["labels"])

[16, 7628, 18, 867, 1220, 49, 53552, 1479, 639, 9079, 1479, 11927, 9, 2, 0]
[417, 892, 6, 4, 1342, 4416, 61, 31, 2690, 5, 4, 13596, 9, 2, 0]


## Metrics

In [ ]:
!python3 -m pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 4.1 MB/s eta 0:00:00


In [ ]:
from datasets import load_metric

metric = load_metric("sacrebleu")

<ipython-input-34-a15d7a2cd8c7>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


In [ ]:
import numpy as np

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # 모델이 예측 로짓(logits)외에 다른 것을 리턴하는 경우.
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # -100은 건너뛴다.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # 단순 후처리
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

## Trainer

In [ ]:
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 6.4 MB/s eta 0:00:00


In [ ]:
import accelerate

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    "test-trainer",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,1.648400
1000,1.380800
1500,1.255700
2000,1.218000
2500,1.146700
3000,1.116500
3500,1.070600
4000,1.070100
4500,1.018900
5000,1.023200


TrainOutput(global_step=49647, training_loss=0.7907866114664551, metrics={'train_runtime': 4673.134, 'train_samples_per_second': 169.98, 'train_steps_per_second': 10.624, 'total_flos': 4834814149656576.0, 'train_loss': 0.7907866114664551, 'epoch': 3.0})

In [ ]:
trainer.save_model("./results")

## Test

In [ ]:
# eval_dataset을 test로 설정
trainer = Seq2SeqTrainer(
    model,
    training_args,
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.evaluate(max_length=max_trg_length)

{'eval_loss': 0.6830718517303467,
 'eval_bleu': 51.02922435564317,
 'eval_runtime': 715.3461,
 'eval_samples_per_second': 23.134,
 'eval_steps_per_second': 1.447}

- 여기서 train.evaluate()를 이용해서 Test 할 때는 bleu socre가 약 51 정도로 나옴

## pipeline

In [ ]:
from transformers import pipeline

result_model_checkpoint = "./results"
translator = pipeline("translation", model=result_model_checkpoint)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


## Inference

In [ ]:
sen_list = [
'모든 액체 , 젤 , 에어로졸 등 은 1 커트 짜리 여닫이 투명 봉지 하나 에 넣 어야 하 ㅂ니다 .',
'미안 하 지만 , 뒷쪽 아이 들 의 떠들 는 소리 가 커 어서 , 광화문 으로 가 아고 싶 은데 표 를 바꾸 어 주 시 겠 어요 ?',
'은행 이 너무 멀 어서 안 되 겠 네요 . 현찰 이 필요 하면 돈 을 훔치 시 어요',
'아무래도 분실 하 ㄴ 것 같 으니 분실 신고서 를 작성 하 아야 하 겠 습니다 . 사무실 로 같이 가 시 ㄹ 까요 ?',
'부산 에서 코로나 확진자 가 급증 하 아서 병상 이 부족하 아 지자  확진자 20명 을 대구 로 이송하 ㄴ다 .',
'변기 가 막히 었 습니다 .',
'그 바지 좀 보이 어 주 시 ㅂ시오 . 이거 얼마 에 사 ㄹ 수 있 는 것 이 ㅂ니까 ?',
'비 가 오 아서 백화점 으로 가지 말 고 두타 로 가 았 으면 좋 겠 습니다 .',
'속 이 안 좋 을 때 는 죽 이나 미음 으로 아침 을 대신 하 ㅂ니다',
'문 대통령 은 집단 이익 에서 벗어 나 아 라고 말 하 었 다 .',
'이것 좀 먹어 보 ㄹ 몇 일 간 의 시간 을 주 시 어요 .',
'이날 개미군단 은 외인 의 물량 을 모두 받 아 내 었 다 .',
'통합 우승 의 목표 를 달성하 ㄴ NC 다이노스 나성범 이 메이저리그 진출 이라는 또 다른 꿈 을 향하 어 나아가 ㄴ다 .',
'이번 구조 조정 이 제품 을 효과 적 으로 개발 하 고 판매 하 기 위하 ㄴ 회사 의 능력 강화 조처 이 ㅁ 을 이해 하 아 주 시 리라 생각 하 ㅂ니다 .',
'요즘 이 프로그램 녹화 하 며 많은 걸 느끼 ㄴ다 ']

In [ ]:
for sent in sen_list:
    print(sent)
    print(translator(sent))
    print()

모든 액체 , 젤 , 에어로졸 등 은 1 커트 짜리 여닫이 투명 봉지 하나 에 넣 어야 하 ㅂ니다 .
[{'translation_text': 'All liquids , gels , aerosols , and so on must be placed in a single one-centered and clear plastic bag .'}]

미안 하 지만 , 뒷쪽 아이 들 의 떠들 는 소리 가 커 어서 , 광화문 으로 가 아고 싶 은데 표 를 바꾸 어 주 시 겠 어요 ?
[{'translation_text': "I 'm sorry , but the noise of the children behind me is loud . I 'd like to go to the Gwanghwamun . Could you change my ticket ?"}]

은행 이 너무 멀 어서 안 되 겠 네요 . 현찰 이 필요 하면 돈 을 훔치 시 어요
[{'translation_text': "I 'm afraid the bank is too far . If you need cash , just steal the money ."}]

아무래도 분실 하 ㄴ 것 같 으니 분실 신고서 를 작성 하 아야 하 겠 습니다 . 사무실 로 같이 가 시 ㄹ 까요 ?
[{'translation_text': "I 'm afraid I 'm afraid I 'm afraid I have to fill out the lost and found form . Would you like to come with me to the office ?"}]

부산 에서 코로나 확진자 가 급증 하 아서 병상 이 부족하 아 지자  확진자 20명 을 대구 로 이송하 ㄴ다 .
[{'translation_text': 'In Busan there is a sharp rise in the number of Koronaeongjins , so that 20 people are sure to be transferred to Daegu when

## Bleu Score

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(result_model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(result_model_checkpoint)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
from torch.utils.data import DataLoader

test_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=64, collate_fn=data_collator
)

In [ ]:
test_dataloader_iter = iter(test_dataloader)

In [ ]:
test_batch = next(test_dataloader_iter)

In [ ]:
test_batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [ ]:
test_input = { key: test_batch[key] for key in ('input_ids', 'attention_mask') }

In [ ]:
english = model.generate(
    **test_input,
    max_length=max_trg_length,
    num_beams=5,
)

In [ ]:
import numpy as np

labels =  np.where(test_batch.labels != -100, test_batch.labels, tokenizer.pad_token_id)
kor_sents = tokenizer.batch_decode(test_batch.input_ids, skip_special_tokens=True) # 원래 문장 리스트
references = tokenizer.batch_decode(labels, skip_special_tokens=True) # 번역 정답 문장 리스트
preds = tokenizer.batch_decode(english, skip_special_tokens=True ) # 번역한 리스트

In [ ]:
# 번역 결과 출력
for s in zip(kor_sents, references, preds):
    print('Korean   :', s[0])
    print('Reference :', s[1])
    print('Translated:', s[2])
    print('\n')

Korean   : ▁그럼▁다른▁데▁가 아서▁먹 어야▁하 겠 네요
Reference : Then I must eat somewhere else!
Translated: Well, maybe we should go somewhere else to eat.


Korean   : ▁이것▁은▁얼마▁이 죠
Reference : How much does it cost?
Translated: How much is this?


Korean   : ▁거리▁가▁보이▁는▁방▁이▁면▁좋 겠 네요
Reference : I 'd like a room looking out on the street.
Translated: I 'd like a room with a view of the street.


Korean   : ▁부르▁시 었 어요
Reference : You wanted to see me?
Translated: Did you call me?


Korean   : ▁변명▁하 려고▁하▁지▁말▁시 어요
Reference : Don't try and explain it away.
Translated: Don't make excuses.


Korean   : ▁미국▁에서▁결혼▁하▁시 ᄅ▁것▁이 ᄇ니까
Reference : Will you get married in America?
Translated: Are you going to get married in America?


Korean   : ▁너▁농담▁하▁니
Reference : You're kidding?
Translated: Are you kidding?


Korean   : ▁저▁는▁중국▁베이징▁에서▁오 았 어요▁당신▁은▁요
Reference : I'm from Beijing, China. How about you?
Translated: I'm from Beijing, China. And you?


Korean   : ▁우리▁는▁런던▁행연결▁비행기▁편▁을▁타 려고▁내리 었 어요
Reference : We're havi

In [ ]:
compute_metrics((english, labels))

{'bleu': 58.85482443144882}

- 직접 test_dataloader를 배치사이즈 64로 만든 후, model.generate를 통해 번역 문장을 생성한 뒤 compute_metrics 함수로 bleu score를 구하면 약 59 정도가 나옴

# 2. KETI_AIR_Downstream_long_ke_t5_base_translation_aihub_ko2en

- 데이터셋 구축 단계까지는 위의 코드와 동일함

## 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 파일 불러오기

In [ ]:
PATH = "/content/drive/MyDrive/"

In [ ]:
ko_path = PATH+"ko-en.ko.parse"
en_path = PATH+"ko-en.en.parse.syn"

In [ ]:
import pandas as pd

In [ ]:
ko_lines = ""
with open(ko_path, "r") as ko_file:
    for line in ko_file.readlines():
        ko_lines += line

In [ ]:
with open(en_path, "r") as en_file:
    en_lines = en_file.readlines()

## 데이터 전처리 (깔끔한 문장의 리스트로 만들기)

In [ ]:
from nltk import Tree

In [ ]:
# nltk의 Tree 모듈을 사용하여 필요한 정보 추출
full_en_text_list = []
for line in en_lines:
    sent = ""
    t = Tree.fromstring(line)
    for token in t.leaves():
      sent += token + ' '
    full_en_text_list.append(sent)

In [ ]:
full_en_text_list[:10]

['Flight 007 will stay on the ground for one hour . ',
 'Flight 017 will stay on the ground for three hours . ',
 "I need 1,000 dollars in traveler 's checks . ",
 'The official exchange rate is around 1,250 Won . ',
 'Please give me three hundred dollar bills and twenty dollar bills for the rest . ',
 'Can I have one hundred dollar bill and four fifty dollar bills ? ',
 'Do you have change for $ 100 ? ',
 "I 'd like to change 100 dollars . ",
 "I 'd like to change $ 100 . ",
 'Change 100 dollars . ']

In [ ]:
# 한 문장씩 나눈 리스트 만들기
ko_list = ko_lines.split("</id>")
for i in range(len(ko_list)):
    ko_list[i] = ko_list[i].split("\n")

ko_list[0]

['<id 1>',
 '<sent 1>',
 '1\t2\tNP\t777/SN',
 '2\t6\tNP_SBJ\t항공편/NNG|은/JX',
 '3\t4\tNP\t1/SN|시간/NNG',
 '4\t6\tNP_AJT\t동안/NNG',
 '5\t6\tNP_AJT\t지상/NNG|에/JKB',
 '6\t7\tVP\t머물/VV|게/EC',
 '7\t0\tVP\t되/VV|ㅂ니다/EF|./SF',
 '</sent>',
 '']

In [ ]:
# 필요한 정보만 추출
import re

pattern = r"[가-힣ㄱ-ㅎ]+|[0-9]+(?=\/SN)"

for i in range(len(ko_list)):
    for j in range(len(ko_list[i])):
        ko_list[i][j] = re.findall(pattern, ko_list[i][j])

In [ ]:
ko_list[:10]

[[[],
  [],
  ['777'],
  ['항공편', '은'],
  ['1', '시간'],
  ['동안'],
  ['지상', '에'],
  ['머물', '게'],
  ['되', 'ㅂ니다'],
  [],
  []],
 [[],
  [],
  [],
  [],
  ['777'],
  ['항공편', '은'],
  ['3', '시간'],
  ['동안'],
  ['지상', '에'],
  ['있', '겠', '습니다'],
  [],
  []],
 [[],
  [],
  [],
  [],
  ['1', '000', '달러'],
  ['여행자', '수표', '가'],
  ['필요', '하', 'ㅂ니다'],
  [],
  []],
 [[], [], [], [], ['1', '250', '원', '이'], ['공식'], ['환율', '이', 'ㅂ니다'], [], []],
 [[],
  [],
  [],
  [],
  ['100', '달러'],
  ['3', '장', '과'],
  ['나머지', '는'],
  ['20', '달러', '권', '으로'],
  ['주', '시', 'ㅂ시오'],
  [],
  []],
 [[],
  [],
  [],
  [],
  ['100', '달러'],
  ['한'],
  ['장', '과'],
  ['50', '달러'],
  ['4', '장', '으로'],
  ['바꾸', '어'],
  ['주', '시', '겠', '어요'],
  [],
  []],
 [[],
  [],
  [],
  [],
  ['100', '달러', '를'],
  ['바꾸', '어'],
  ['주', '시', '겠', '어요'],
  [],
  []],
 [[], [], [], [], ['100', '달러', '만'], ['바꾸', '어'], ['주', '시', '어요'], [], []],
 [[],
  [],
  [],
  [],
  ['100', '달러', '만'],
  ['환전'],
  ['좀'],
  ['하', '아'],
  ['주', '시', '어요'],
  []

In [ ]:
# 문장으로 만들어서 리스트에 넣어주기
ko_text_list = []

for i in range(len(ko_list)-1): # id로 split 해서 1개 추가로 생긴 부분 빼줌
    full_sent = ""
    for j in range(len(ko_list[i])):
        if ko_list[i][j]:
            for token in ko_list[i][j]:
                full_sent += token + ' '

    ko_text_list.append(full_sent)

In [ ]:
ko_text_list[:10]

['777 항공편 은 1 시간 동안 지상 에 머물 게 되 ㅂ니다 ',
 '777 항공편 은 3 시간 동안 지상 에 있 겠 습니다 ',
 '1 000 달러 여행자 수표 가 필요 하 ㅂ니다 ',
 '1 250 원 이 공식 환율 이 ㅂ니다 ',
 '100 달러 3 장 과 나머지 는 20 달러 권 으로 주 시 ㅂ시오 ',
 '100 달러 한 장 과 50 달러 4 장 으로 바꾸 어 주 시 겠 어요 ',
 '100 달러 를 바꾸 어 주 시 겠 어요 ',
 '100 달러 만 바꾸 어 주 시 어요 ',
 '100 달러 만 환전 좀 하 아 주 시 어요 ',
 '100 달러 만 환전 하 아 주 시 어요 ']

In [ ]:
print(ko_text_list[:10])
print(full_en_text_list[:10])

print(len(ko_text_list))
print(len(full_en_text_list))

['777 항공편 은 1 시간 동안 지상 에 머물 게 되 ㅂ니다 ', '777 항공편 은 3 시간 동안 지상 에 있 겠 습니다 ', '1 000 달러 여행자 수표 가 필요 하 ㅂ니다 ', '1 250 원 이 공식 환율 이 ㅂ니다 ', '100 달러 3 장 과 나머지 는 20 달러 권 으로 주 시 ㅂ시오 ', '100 달러 한 장 과 50 달러 4 장 으로 바꾸 어 주 시 겠 어요 ', '100 달러 를 바꾸 어 주 시 겠 어요 ', '100 달러 만 바꾸 어 주 시 어요 ', '100 달러 만 환전 좀 하 아 주 시 어요 ', '100 달러 만 환전 하 아 주 시 어요 ']
['Flight 007 will stay on the ground for one hour . ', 'Flight 017 will stay on the ground for three hours . ', "I need 1,000 dollars in traveler 's checks . ", 'The official exchange rate is around 1,250 Won . ', 'Please give me three hundred dollar bills and twenty dollar bills for the rest . ', 'Can I have one hundred dollar bill and four fifty dollar bills ? ', 'Do you have change for $ 100 ? ', "I 'd like to change 100 dollars . ", "I 'd like to change $ 100 . ", 'Change 100 dollars . ']
330974
330974


## 데이터셋 구축

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.4 MB/s eta 0:00:00


In [ ]:
from datasets import DatasetDict, Dataset, Features, Value

data_dict = {
    'id': range(1, len(ko_text_list) + 1),
    'translation': [{'ko': ko, 'en': en} for ko, en in zip(ko_text_list, full_en_text_list)]
}

features = Features({
    'id': Value('int32'),
    'translation': {'ko': Value('string'), 'en': Value('string')}
})

raw_datasets = Dataset.from_dict(data_dict, features=features)
dataset = DatasetDict({'train': raw_datasets})

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 330974
    })
})


In [ ]:
split_datasets = dataset["train"].train_test_split(train_size=0.8, test_size=0.1, seed=20)

dataset["train"] = split_datasets["train"]
dataset["validation"] = split_datasets["test"].train_test_split(train_size=0.5, seed=20)["train"]
dataset["test"] = split_datasets["test"].train_test_split(train_size=0.5, seed=20)["test"]

dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 264779
    })
    validation: Dataset({
        features: ['id', 'translation'],
        num_rows: 16549
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 16549
    })
})

## 데이터 전처리 (tokenize)

In [ ]:
!pip install transformers==4.28.0 # PartialState Error를 방지하기 위해 4.28.0 버전 사용
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 76.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is 

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "KETI-AIR-Downstream/long-ke-t5-base-translation-aihub-ko2en"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
dataset['train'][0]['translation']['ko'], dataset['train'][0]['translation']['en']

('즉석 에서 투표 하 자 ', "Let 's vote before we speak another word . ")

In [ ]:
max_src_length = 64
max_trg_length = 64

def tokenize_function(examples):
    src = [ex["ko"] for ex in examples["translation"]]
    trg = [ex["en"] for ex in examples["translation"]]
    model_inputs = tokenizer(src, text_target=trg, max_length=max_src_length, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(trg, max_length=max_trg_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [ ]:
tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

Map:   0%|          | 0/264779 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/16549 [00:00<?, ? examples/s]

Map:   0%|          | 0/16549 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 264779
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 16549
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 16549
    })
})

## Model

In [ ]:
import torch

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

## Data Collation

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
batch.keys()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [ ]:
batch

{'input_ids': tensor([[  122,  4059,  4149, 19416,  1202,    58,   361,   289,  1022,   112,
           262,  1804,  1908, 20004,     1],
        [ 6280,  3350,   462, 20283,  8340,   289,  6507,    88, 20004,     2,
           513,     7, 20004,     1,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]]), 'labels': tensor([[20027, 22432, 20025, 20533, 20004, 20272, 20049, 53359, 35681, 20388,
         22157, 24859, 20004, 20005, 20004,     1],
        [20283, 20937, 20010, 20007, 20467, 21537, 20044, 20032, 20786, 20009,
         20007, 22517, 20004, 20005, 20004,     1]]), 'decoder_input_ids': tensor([[    0, 20027, 22432, 20025, 20533, 20004, 20272, 20049, 53359, 35681,
         20388, 22157, 24859, 20004, 20005, 20004],
        [    0, 20283, 20937, 20010, 20007, 20467, 21537, 20044, 20032, 20786,
         20009, 20007, 22517, 20004, 20005, 20004]])}

In [ ]:
batch["labels"]

tensor([[20027, 22432, 20025, 20533, 20004, 20272, 20049, 53359, 35681, 20388,
         22157, 24859, 20004, 20005, 20004,     1],
        [20283, 20937, 20010, 20007, 20467, 21537, 20044, 20032, 20786, 20009,
         20007, 22517, 20004, 20005, 20004,     1]])

In [ ]:
batch["decoder_input_ids"]

tensor([[    0, 20027, 22432, 20025, 20533, 20004, 20272, 20049, 53359, 35681,
         20388, 22157, 24859, 20004, 20005, 20004],
        [    0, 20283, 20937, 20010, 20007, 20467, 21537, 20044, 20032, 20786,
         20009, 20007, 22517, 20004, 20005, 20004]])

In [ ]:
for i in range(1, 3):
    print(tokenized_datasets["train"][i]["labels"])

[20027, 22432, 20025, 20533, 20004, 20272, 20049, 53359, 35681, 20388, 22157, 24859, 20004, 20005, 20004, 1]
[20283, 20937, 20010, 20007, 20467, 21537, 20044, 20032, 20786, 20009, 20007, 22517, 20004, 20005, 20004, 1]


## Metrics

In [ ]:
!python3 -m pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 3.8 MB/s eta 0:00:00


In [ ]:
from datasets import load_metric

metric = load_metric("sacrebleu")

<ipython-input-35-a15d7a2cd8c7>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


In [ ]:
import numpy as np

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # 모델이 예측 로짓(logits)외에 다른 것을 리턴하는 경우.
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # -100은 건너뛴다.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # 단순 후처리
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

## Trainer

In [ ]:
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 6.6 MB/s eta 0:00:00


In [ ]:
import accelerate

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    "test-trainer_model_2",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1, # 시간이 너무 오래 걸리는 관계로 epoch 1만 훈련함.
    predict_with_generate=True,
    gradient_accumulation_steps=2,
    fp16=False,
)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:816: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Step,Training Loss
500,1.645300
1000,1.226200
1500,1.153600
2000,1.106000
2500,1.069500
3000,1.063700
3500,1.044100
4000,1.026600
4500,1.030700
5000,1.015600


TrainOutput(global_step=8274, training_loss=1.0791916841465925, metrics={'train_runtime': 7219.9644, 'train_samples_per_second': 36.673, 'train_steps_per_second': 1.146, 'total_flos': 9813119581126656.0, 'train_loss': 1.0791916841465925, 'epoch': 1.0})

In [ ]:
trainer.save_model("./results_model_2")

## Test

In [ ]:
# eval_dataset을 test로 설정
trainer = Seq2SeqTrainer(
    model,
    training_args,
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:816: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


{'eval_loss': 0.8814811110496521,
 'eval_bleu': 35.804538726684655,
 'eval_runtime': 623.9212,
 'eval_samples_per_second': 26.524,
 'eval_steps_per_second': 1.659}

- 여기서 train.evaluate()를 이용해서 Test 할 때는 bleu socre가 약 36 정도로 나옴
- 첫 번째 사전학습모델(Helsinki_NLP_opus_mt_ko_en)보다 낮은 이유는 모델 자체의 다름 요인과 시간 관계상 epoch를 1까지만 훈련했다는 요인 등이 존재할 것임

## pipeline

In [ ]:
from transformers import pipeline

result_model_checkpoint = "./results_model_2"
translator = pipeline("translation", model=result_model_checkpoint)

## Inference

In [ ]:
sen_list = [
'모든 액체 , 젤 , 에어로졸 등 은 1 커트 짜리 여닫이 투명 봉지 하나 에 넣 어야 하 ㅂ니다 .',
'미안 하 지만 , 뒷쪽 아이 들 의 떠들 는 소리 가 커 어서 , 광화문 으로 가 아고 싶 은데 표 를 바꾸 어 주 시 겠 어요 ?',
'은행 이 너무 멀 어서 안 되 겠 네요 . 현찰 이 필요 하면 돈 을 훔치 시 어요',
'아무래도 분실 하 ㄴ 것 같 으니 분실 신고서 를 작성 하 아야 하 겠 습니다 . 사무실 로 같이 가 시 ㄹ 까요 ?',
'부산 에서 코로나 확진자 가 급증 하 아서 병상 이 부족하 아 지자  확진자 20명 을 대구 로 이송하 ㄴ다 .',
'변기 가 막히 었 습니다 .',
'그 바지 좀 보이 어 주 시 ㅂ시오 . 이거 얼마 에 사 ㄹ 수 있 는 것 이 ㅂ니까 ?',
'비 가 오 아서 백화점 으로 가지 말 고 두타 로 가 았 으면 좋 겠 습니다 .',
'속 이 안 좋 을 때 는 죽 이나 미음 으로 아침 을 대신 하 ㅂ니다',
'문 대통령 은 집단 이익 에서 벗어 나 아 라고 말 하 었 다 .',
'이것 좀 먹어 보 ㄹ 몇 일 간 의 시간 을 주 시 어요 .',
'이날 개미군단 은 외인 의 물량 을 모두 받 아 내 었 다 .',
'통합 우승 의 목표 를 달성하 ㄴ NC 다이노스 나성범 이 메이저리그 진출 이라는 또 다른 꿈 을 향하 어 나아가 ㄴ다 .',
'이번 구조 조정 이 제품 을 효과 적 으로 개발 하 고 판매 하 기 위하 ㄴ 회사 의 능력 강화 조처 이 ㅁ 을 이해 하 아 주 시 리라 생각 하 ㅂ니다 .',
'요즘 이 프로그램 녹화 하 며 많은 걸 느끼 ㄴ다 ']

In [ ]:
for sent in sen_list:
    print(sent)
    print(translator(sent))
    print()

Your input_length: 31 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


모든 액체 , 젤 , 에어로졸 등 은 1 커트 짜리 여닫이 투명 봉지 하나 에 넣 어야 하 ㅂ니다 .


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:816: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
Your input_length: 44 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


[{'translation_text': 'All liquids , gels , aerosols , etc. must be placed in a'}]

미안 하 지만 , 뒷쪽 아이 들 의 떠들 는 소리 가 커 어서 , 광화문 으로 가 아고 싶 은데 표 를 바꾸 어 주 시 겠 어요 ?


Your input_length: 26 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


...

[{'translation_text': 'President Moon said to get out of the collective interest . '}]

이것 좀 먹어 보 ㄹ 몇 일 간 의 시간 을 주 시 어요 .
[{'translation_text': 'Give me a few days to try this . '}]

이날 개미군단 은 외인 의 물량 을 모두 받 아 내 었 다 .


Your input_length: 36 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


[{'translation_text': 'On this day, the Antsudaeundan received all the supplies of foreign '}]

통합 우승 의 목표 를 달성하 ㄴ NC 다이노스 나성범 이 메이저리그 진출 이라는 또 다른 꿈 을 향하 어 나아가 ㄴ다 .


Your input_length: 43 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


[{'translation_text': 'NC Dinos Na Seong-beom, who has achieved the goal of a unified championship'}]

이번 구조 조정 이 제품 을 효과 적 으로 개발 하 고 판매 하 기 위하 ㄴ 회사 의 능력 강화 조처 이 ㅁ 을 이해 하 아 주 시 리라 생각 하 ㅂ니다 .
[{'translation_text': "I think you understand that this restructuring is a company's ability-building measure to develop and"}]

요즘 이 프로그램 녹화 하 며 많은 걸 느끼 ㄴ다 
[{'translation_text': 'These days I feel a lot while recording this program . '}]



## Bleu Score

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(result_model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(result_model_checkpoint)

In [ ]:
from torch.utils.data import DataLoader

test_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=64, collate_fn=data_collator
)

In [ ]:
test_dataloader_iter = iter(test_dataloader)

In [ ]:
test_batch = next(test_dataloader_iter)

In [ ]:
test_batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [ ]:
test_input = { key: test_batch[key] for key in ('input_ids', 'attention_mask') }

In [ ]:
english = model.generate(
    **test_input,
    max_length=max_trg_length,
    num_beams=5,
)

In [ ]:
import numpy as np

labels =  np.where(test_batch.labels != -100, test_batch.labels, tokenizer.pad_token_id)
kor_sents = tokenizer.batch_decode(test_batch.input_ids, skip_special_tokens=True) # 원래 문장 리스트
references = tokenizer.batch_decode(labels, skip_special_tokens=True) # 번역 정답 문장 리스트
preds = tokenizer.batch_decode(english, skip_special_tokens=True ) # 번역한 리스트

In [ ]:
# 번역 결과 출력
for s in zip(kor_sents, references, preds):
    print('Korean   :', s[0])
    print('Reference :', s[1])
    print('Translated:', s[2])
    print('\n')

Korean   : 그럼 다른 데 가 아서 먹 어야 하 겠 네요 
Reference : Then I must eat somewhere else! 
Translated: Then I 'll have to go somewhere else. 


Korean   : 이것 은 얼마 이 죠 
Reference : How much does it cost? 
Translated: How much is this? 


Korean   : 거리 가 보이 는 방 이 면 좋 겠 네요 
Reference : I 'd like a room looking out on the street. 
Translated: I 'd like a room with a street view. 


Korean   : 부르 시 었 어요 
Reference : You wanted to see me? 
Translated: Did you call me? 


Korean   : 변명 하 려고 하 지 말 시 어요 
Reference : Don't try and explain it away. 
Translated: Don't try to make excuses. 


Korean   : 미국 에서 결혼 하 시 ᄅ 것 이 니까 
Reference : Will you get married in America? 
Translated: Are you getting married in America? 


Korean   : 너 농담 하 니 
Reference : You're kidding? 
Translated: Are you kidding me? 


Korean   : 저 는 중국 베이징 에서 오 았 어요 당신 은 요 
Reference : I'm from Beijing, China. How about you? 
Translated: I'm from Beijing, China. You? 


Korean   : 우리 는 런던 행연결 비행기 편 을 타 려고 내리 었 어요 
Reference : We're havin

In [ ]:
compute_metrics((english, labels))

{'bleu': 37.422200547040475}

- 직접 test_dataloader를 배치사이즈 64로 만든 후, model.generate를 통해 번역 문장을 생성한 뒤 compute_metrics 함수로 bleu score를 구하면 약 37 정도가 나옴
- 첫 번째 사전학습모델(Helsinki_NLP_opus_mt_ko_en)보다 낮은 이유는 모델 자체의 다름 요인과 시간 관계상 epoch를 1까지만 훈련했다는 요인 등이 존재할 것임

# 결과 정리

- Inference
    - 두 모델 모두 잘 나오는 것으로 보이는데, 그래도 Helsinki_NLP_opus_mt_ko_en이 조금 더 정확하게 번역하는 것 같다.

- Bleu Score
    - Helsinki_NLP_opus_mt_ko_en: 59
    - KETI_AIR_Downstream_long_ke_t5_base_translation_aihub_ko2en: 37



